In [1]:
#Importação das bibliotecas
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import urllib

In [36]:
#Adequando o dataframe com os avisos de procedimentos e exames
df_aviso = pd.read_excel('CONTROLE AVISO.xlsx')
df_aviso.head()

,Nome,Telefone,Procedimento,Data/Hora,Controle
0,Fulano da Silva,(13) 00000-0000,Ultrassonografia De Globo Ocular / Orbita (Mon...,20/08/2024 - 16:40,Sem sucesso
1,Ciclano Souza,NaN,Tomografia de Cranio C/ Contraste (PMV),22/08/2024 - 07:00,Sem sucesso
2,Beltrano Almeida,(13) 99999-9999,Consulta Pre-Natal Alto Risco (PMV),10/09/2024 - 08:40,Sem sucesso
3,Fulano da Silva,(13) 99999-9999,Consulta Pre-Natal Alto Risco (PMV),17/09/2024 - 08:00,Sem sucesso
4,Ciclano Souza,(13) 99999-9999,Eletroneuromiograma Membro Superior Direito(En...,19/09/2024 - 07:35,Sem sucesso


In [ ]:
#Abre o navegador para logar o whatsapp
driver = webdriver.Chrome()
driver.get('https://web.whatsapp.com/')
sleep(10)

#Loop para logar o captcha
while True:
    try:
        campo_busca = driver.find_element(By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]')
        break #whatsapp já está logado

    except NoSuchElementException:
        print("Whatsapp ainda não foi logado, nova tentativa em 10 segundos")
        sleep(10)

In [ ]:
#Envia as mensagens para cada paciente na listagem
for i in df_aviso.index:
        if df_aviso.loc[i,'Controle'] != 'Enviado':
                mensagem = f'''Olá, Sr(a) *{df_aviso.loc[i,'Nome']}*
Estamos entrando em contato para comunicar o agendamento de: 
*{df_aviso.loc[i,'Procedimento']}*
*{df_aviso.loc[i,'Data/Hora']}*

Digite *1* Para *CONFIRMAR*
Digite *2* Para *REMARCAR* (Justifique o motivo)
Digite *3* Para *CONFIRMAR* (Justifique o motivo)

at.te
'''
                contato = df_aviso.loc[i,'Telefone']
                if type(contato) == float: #Garante que a aplicação continua funcionando com valores nulos
                        df_aviso.loc[i,'Controle'] = 'Sem sucesso'
                        continue
                texto = urllib.parse.quote(mensagem)
                try:
                        driver.get(f'https://web.whatsapp.com/send/?phone=55${contato}&text={texto}')
                        
                        while len(driver.find_elements(By.ID,'side')) < 1:
                                sleep(1)
                        sleep(5)
                        driver.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p').send_keys(Keys.ENTER)
                        sleep(5)
                        df_aviso.loc[i,'Controle'] = 'Enviado'
                except NoSuchElementException: #Exceção para quando não existe whatsapp cadastrado no número
                        df_aviso.loc[i,'Controle'] = 'Sem sucesso'
                        continue
        else:
                continue

In [35]:
#Sobrescreve o DataFrame na planilha para atualização dos contatos inexistentes
df_aviso.to_excel('CONTROLE AVISO.xlsx',index=False)